In [ ]:
# !pip install langchain langchain-openai langchain-cohere langchain-community chromadb
# !pip install -U langchain langchain-openai langchain-community langchain-cohere chromadb sentence-transformers python-dotenv
# ! pip install langchain_classic
# %pip install -U langchain langchain-community langchain-openai langchain-cohere chromadb

# PyTorch 핵심 라이브러리 설치
# !pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# NLP 라이브러리 설치
# !pip install --no-cache-dir transformers sentence-transformers

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import json
import torch
from typing import List, Any, Dict
from dotenv import load_dotenv

# LangChain 및 관련 모듈
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereRerank
from sentence_transformers import SentenceTransformer

# 환경 변수 로드
load_dotenv()

True

In [4]:
import json
from typing import List, Any, Dict
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import SentenceTransformer

class GemmaEmbeddings:
    def __init__(self, model_path: str):
        self.model = SentenceTransformer(model_path)
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode([f"title: none | text: {text}" for text in texts]).tolist()
    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(f"task: search result | query: {text}").tolist()

class CardANDRetriever(BaseRetriever):

    vectorstore: Any
    card_map: Dict
    intent_extractor: Any
    search_depth: int = 200

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1. 동적 의도 추출 (LLM 활용)
        keywords_json = self.intent_extractor.invoke({"question": query})
        try:
            search_intents = json.loads(keywords_json.replace("```json", "").replace("```", ""))
        except:
            search_intents = [query]
        print(f"추출된 키워드: {search_intents}")

        # 2. 각 의도별 검색 및 점수 수집 (원본 Step 4 - 1단계)
        intent_scores = []
        for intent in search_intents:
            results = self.vectorstore.similarity_search_with_relevance_scores(intent, k=self.search_depth)
            # ID를 키로, 점수를 값으로 저장
            current_hits = {str(doc.metadata['card_id']): score for doc, score in results}
            intent_scores.append(current_hits)

        # 3. 교집합(AND) 필터링 (원본 Step 4 - 2단계)
        common_ids = set(intent_scores[0].keys())
        for hits in intent_scores[1:]:
            common_ids &= set(hits.keys())
        
        if not common_ids:
            common_ids = set(list(intent_scores[0].keys())[:20])

        # 4. 점수 합산 및 데이터 복원 (원본 Step 4 - 3단계)
        final_docs = []
        for c_id in list(common_ids):
            # 종합 점수 계산: $$Score_{total} = \sum Score_{intent}$$
            total_score = sum(intent_map[c_id] for intent_map in intent_scores if c_id in intent_map)
            
            card = self.card_map.get(str(c_id))
            if not card: continue
            
            content = f"카드명: {card['name']} | 상세혜택: {' / '.join(card['full_details'])} | 조건: {card['structured'].get('condition', '')}"
            final_docs.append(Document(
                page_content=content, 
                metadata={"total_score": total_score, **card}
            ))
            
        # 5. 최종 정렬 (원본 Step 4 - 4단계)
        return sorted(final_docs, key=lambda x: x.metadata['total_score'], reverse=True)

In [68]:
class CardConciergeRAG:
    def __init__(self, model_path, db_path, data_path):
        self.embeddings = GemmaEmbeddings(model_path)
        self.db_path = db_path
        self.data_path = data_path
        
        self.card_map = self._setup_card_map()
        self.vectorstore = Chroma(
            collection_name="card_benefits",  
            persist_directory=db_path, 
            embedding_function=self.embeddings
        )
        
        self._check_and_index()
        self.chain = self._build_chain()

    def _setup_card_map(self):
        """데이터 병합 및 '리스트 속 리스트' 방지 로직이 적용된 마스터 맵 생성"""
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        mapping = {}
        for item in data:
            meta = item['metadata']
            c_id = str(meta['card_id'])
            ai_s = item['ai_structured']
            
            if c_id not in mapping:
                mapping[c_id] = {
                    "name": meta['card_name'], 
                    "corp": meta['corp'],
                    "metadata": meta, 
                    "full_details": [item['content']], 
                    "structured": {}
                }
                # 모든 필드 초기화 (리스트로 통일)
                for k, v in ai_s.items():
                    if v and v != "정보 없음":
                        mapping[c_id]["structured"][k] = list(v) if isinstance(v, list) else [v]
                    else:
                        mapping[c_id]["structured"][k] = []
            else:
                mapping[c_id]["full_details"].append(item['content'])
                target_s = mapping[c_id]["structured"]
                
                for k, v in ai_s.items():
                    if v and v != "정보 없음":
                        if k not in target_s: target_s[k] = []
                        if isinstance(v, list):
                            # 중복 제거하며 병합
                            target_s[k].extend([x for x in v if x not in target_s[k]])
                        elif isinstance(v, dict):
                            # 삼성카드와 같은 딕셔너리 데이터 처리
                            dict_str = ", ".join([f"{key}: {val}" for key, val in v.items()])
                            if dict_str not in target_s[k]: target_s[k].append(dict_str)
                        else:
                            if v not in target_s[k]: target_s[k].append(v)
        return mapping

    def _check_and_index(self):
        current_count = self.vectorstore._collection.count()
        if current_count == 0:
            print("ChromaDB 인덱싱 시작...")
            with open(self.data_path, 'r', encoding='utf-8') as f:
                master_data = json.load(f)
            docs = []
            for item in master_data:
                metadata = item['metadata'].copy()
                metadata['card_id'] = str(metadata['card_id'])
                docs.append(Document(page_content=item['embedding_input'], metadata=metadata))
            batch_size = 500
            for i in range(0, len(docs), batch_size):
                self.vectorstore.add_documents(docs[i : i + batch_size])
            print(f"인덱싱 완료. (총 {len(docs)}건)")
        else:
            print(f"기존 DB 로드 완료 (데이터: {current_count}건)")

    def _build_chain(self):
# 1. 의도 추출기 (데이터베이스 카테고리 매칭 최적화)

# 1. 의도 추출기 (성별, 나이 등 인구통계 정보 제외 가이드 추가)
        intent_prompt = ChatPromptTemplate.from_template("""
        사용자의 질문을 분석하여, 오직 '카드 혜택'과 관련된 짧고 명확한 검색 문장 리스트만 생성해줘.

        [가이드라인]
        1. **중요: 사용자의 성별, 나이, 직업(예: 20대, 남성, 직장인) 등 인구통계학적 정보는 무시하고 검색어에 포함하지 마세요.**
        2. 단순 명사가 아닌, '혜택 대상 + 행위'가 포함된 짧은 문장으로 만드세요.
        3. 여러 세부 항목은 하나의 대표 카테고리로 통합하세요. (예: 관리비, 가스 요금 -> "공과금 할인 혜택")
        4. 브랜드 이름보다는 혜택의 종류를 명확히 기술하세요. (예: 배민/쿠팡이츠 -> "배달 앱 푸드 할인")
        5. ChromaDB 임베딩 매칭을 위해 '신용카드/체크카드' 여부도 혜택 문장에서 제외하고 순수 혜택 내용만 추출하세요.

        [예시]
        질문: "20대 남성 직장인인데 대중교통 할인되고 관리비 아낄 수 있는 체크카드 추천해줘."
        응답: ["버스 지하철 등 대중교통 할인", "아파트 관리비 및 공과금 혜택"]

        질문: {question}
        형식: ["문장1", "문장2", ...] (JSON 리스트로만 응답하세요)
        """)
        
        intent_extractor = (
            intent_prompt 
            | ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, model_kwargs={"top_p": 0.9}) 
            | StrOutputParser()
        )

        base_retriever = CardANDRetriever(
            vectorstore=self.vectorstore, card_map=self.card_map, 
            intent_extractor=intent_extractor, search_depth=200
        )
        
        compressor = CohereRerank(model="rerank-v3.5", top_n=5)
        retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)

        # 3. 답변 생성 프롬프트

        prompt = ChatPromptTemplate.from_template("""
        당신은 대한민국 최고의 '신용카드 비교 분석 전문가'입니다. 
        사용자의 요구사항을 **단 하나라도 만족하지 못하는 카드를 1위로 추천하는 것은 절대 용납되지 않습니다.**

        사용자 질문: {question}
        [카드 데이터]: {context}

        [답변 작성 필살 규칙 - 시연 성공을 위한 마지막 경고]
        1. **무조건적인 탈락 (Zero Tolerance)**: 사용자가 요청한 핵심 키워드(예: 관리비, 가스비)가 혜택에 포함되어 있지 않거나 '제외'라고 명시된 카드는 리랭킹 점수가 1등이어도 무조건 추천 리스트에서 삭제하세요. 
        2. **논리적 재정렬**: [B. 원문 전체]를 샅샅이 뒤져서 사용자의 '모든' 조건을 충족하는 유일한 카드를 찾아 1위로 올리세요. (현재 데이터상 LOCA 365가 이에 해당할 확률이 높습니다.)
        3. **1위 카드 (BEST) 상세 분석**: 
           - 반드시 카드 상세 링크를 포함하세요: 🔗 [카드 상세정보 확인하기](상세 링크)
           - 원문에서 찾은 구체적 할인율(%)과 한도를 정확히 기술하세요.
        4. **기타 추천 (2~3위)**: 조건에 부합하는 다른 카드가 있다면 카드명과 핵심만 아주 짧게 언급하세요.
        5. **톤앤매너**: 다정한 전문가 말투 유지, 불렛 포인트 활용. 

        조건에 맞지 않는 카드를 1위로 올리는 실수를 반복하지 마세요.
        
        [단종 의심 주의보] 
        - 최근 카드사의 정책 변화로 인기 카드가 예고 없이 단종되는 경우가 많습니다.
        - 만약 추천하는 카드가 출시된 지 오래되었거나 혜택이 지나치게 좋다면, "현재 시점에서 발급 가능 여부를 반드시 공식 홈페이지에서 재확인하라"는 주의 문구를 답변 하단에 다정하게 포함하세요.
        """)

        # 4. 데이터 구조화 함수 (요약 + 원문 전체 통합 버전)
        def format_docs(docs):
            formatted = []

            for i, doc in enumerate(docs):
                m = doc.metadata
                s = m.get('structured', {})
                raw_meta = m.get('metadata', {})
                card_id = str(raw_meta.get('card_id')) # ID 추출
                
                # [핵심] card_map에서 해당 ID의 모든 원문(full_details)을 가져와 합칩니다.
                card_full_info = self.card_map.get(card_id, {})
                # 리스트로 저장된 여러 카테고리의 원문을 문단으로 구분하여 합침
                all_raw_contents = "\n\n".join(card_full_info.get("full_details", ["원문 정보 없음"]))

                # 방탄 join 로직
                def safe_join(data_list):
                    if not data_list: return "정보 없음"
                    flat = []
                    for item in data_list:
                        if isinstance(item, list): flat.extend([str(x) for x in item])
                        else: flat.append(str(item))
                    return ", ".join(dict.fromkeys(flat))

                # 카드고릴라 링크
                card_link = f"https://www.card-gorilla.com/card/detail/{card_id}" if card_id != 'None' else "링크 없음"

                # LLM에 주입될 초정밀 컨텍스트
                info = (
                    f"### [추천 순위 {i+1}위] {m.get('name')} ({m.get('corp')})\n"
                    f"- 연회비: {raw_meta.get('annual_fee', '정보 없음')}\n"
                    f"- 전월 실적 기준: {raw_meta.get('min_performance', '정보 없음')}원 이상\n"
                    f"- 상세 링크: {card_link}\n"
                    f"\n  [A. 핵심 요약 데이터]\n"
                    f"  ▶ 통합 요약: {safe_join(s.get('summary'))}\n"
                    f"  ▶ 가맹점 목록: {safe_join(s.get('merchants'))}\n"
                    f"  ▶ 실적 유예/조건: {safe_join(s.get('grace_period'))} / {safe_join(s.get('payment_conditions'))}\n"
                    f"  ▶ 제외 항목(혜택/실적): {safe_join(s.get('benefit_exclusions'))} / {safe_join(s.get('performance_exclusions'))}\n"
                    f"  ▶ 기타 고유 정보: {safe_join(s.get('additional_info'))}\n"
                    f"\n  [B. 카드 혜택 데이터 원문 전체]\n"
                    f"{all_raw_contents}\n" 
                    f"==================================================\n"
                )
                
                # print(f"\n✅ {i+1}위 카드 데이터 통합 결과:")
                # print(info)
                # print("-" * 80)
                
                formatted.append(info)
            
            return "\n\n".join(formatted)

        return (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt 
            | ChatOpenAI(model="gpt-4.1-mini", temperature=0.8, max_tokens=1500) 
            # | ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, model_kwargs={"top_p": 0.1}, max_tokens=1500) 
            | StrOutputParser()
        )

    def ask(self, query):
        return self.chain.invoke(query)

In [86]:
concierge = CardConciergeRAG(
    model_path='./models/gemma-300m-4080super-extreme',
    db_path='./data/chroma_db',
    data_path='./data/FINAL_MASTER_DATA_FIXED_7757.json'
)

print(concierge.ask("30대 직장인인데, 지방 출장이 잦아서 전기차 충전 할인이 꼭 필요해. 고속도로를 탈 일이 많다 보니 통행료 할인도 있었으면 좋겠고, 이동 중에 편하게 들르는 드라이브스루 할인까지 지원되는 카드 추천해줘"))

Loading weights:   0%|          | 0/314 [00:00<?, ?it/s]

기존 DB 로드 완료 (데이터: 7757건)


C:\Users\Jeon\AppData\Local\Temp\ipykernel_3724\4272190409.py:1: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  concierge = CardConciergeRAG(


추출된 키워드: ['전기차 충전 할인', '고속도로 통행료 할인', '드라이브스루 할인']
안녕하세요! 30대 직장인 분께서 **지방 출장이 잦아 전기차 충전 할인과 고속도로 통행료 할인, 그리고 드라이브스루 할인**을 원하신다고 하셨는데요. 주어진 데이터를 꼼꼼히 분석한 결과, 모든 조건을 **완벽하게 만족하는 카드는 단 하나**였습니다.

---

## 1위 (BEST) 추천 카드:  
### 삼성 iD ENERGY 카드 (삼성카드)  
🔗 [카드 상세정보 확인하기](https://www.card-gorilla.com/card/detail/2290)

### 이유 및 상세 혜택

- **전기차 충전 할인**:  
  - 환경부, 테슬라, 한국전기차충전서비스, 차지비 등 다양한 전기차 충전 사업자 이용 시  
  - **10% 결제일 할인 제공** (전월 50만원 이상 시)  
  - 전월 50만원 이상 조건을 맞추면 월 5,000원 한도 내 할인 가능

- **고속도로 통행료 할인**:  
  - 하이패스 고속도로 통행료 결제 시  
  - **10% 결제일 할인 제공** (월 최대 5,000원 한도)  
  - 마찬가지로 전월 50만원 이상 시 제공, 발급월+1개월까지는 유예

- **드라이브스루 할인**:  
  - 스타벅스 드라이브스루 매장에서 ‘My DT Pass’ 등록 후 삼성 iD ENERGY 카드 결제 시  
  - **30% 결제일 할인** (월 최대 5,000원 한도)  
  - 전월 50만원 이상 시 제공

- **추가 혜택**:  
  - 주차장 및 대리운전 10% 할인 (월 5,000원 한도)  
  - 차량 정비(엔진오일 교환 등) 할인 및 무료 서비스 제공  
  - 대중교통/택시 10% 할인 (전기차 충전과 통합 월 5,000원 한도)

### 전월 실적 조건  
- 50만원 이상 (발급월+1개월까지는 유예)

---

## 2~3위 추천 카드 간략 안내

- **삼성 iD EV 카드**  
  - 전기차 충전 최대 70% 할인(

In [87]:
# 1. 시연용 페르소나 5종 리스트 정의
personas = [
    "20대 남성 직장인인데, 대중교통 이용이랑 OTT 구독, 그리고 관리비나 가스비 같은 공과금 할인이 다 되는 신용카드 좀 추천해줘.",
    "20대 여성 직장인인데, 퇴근 후 넷플릭스 보는 게 낙이라 OTT 할인이 꼭 필요해. 해외 직구를 많이 해서 결제 수수료도 아끼고 싶고, 온라인 간편결제가 되는 카드를 추천해줘.",
    "10대 고등학생인데, 매일 버스랑 지하철로 등하교해서 대중교통 할인이 꼭 필요해요. OTT구독료도 아끼고 싶고, 온라인 간편결제 할인이 되는 체크카드 추천해주세요.",
    "30대 직장인인데, 지방 출장이 잦아서 전기차 충전 할인이 꼭 필요해. 고속도로를 탈 일이 많다 보니 통행료 할인도 있었으면 좋겠고, 이동 중에 편하게 들르는 드라이브스루 할인까지 지원되는 카드 추천해줘.",
]

# 2. 반복문을 통한 일괄 실행
print("="*30)
print("🚀 Gemma-Bot 페르소나별 맞춤 추천 시연 시작")
print("="*30)

for i, query in enumerate(personas, 1):
    print(f"\n👤 [테스트 페르소나 {i}]")
    print(f"질문: {query}")
    print("-" * 50)
    
    # RAG 실행
    response = concierge.ask(query)
    
    print(response)
    print("\n" + "="*50)

🚀 Gemma-Bot 페르소나별 맞춤 추천 시연 시작

👤 [테스트 페르소나 1]
질문: 20대 남성 직장인인데, 대중교통 이용이랑 OTT 구독, 그리고 관리비나 가스비 같은 공과금 할인이 다 되는 신용카드 좀 추천해줘.
--------------------------------------------------
추출된 키워드: ['대중교통 이용 할인', 'OTT 구독 할인', '공과금 할인 혜택']
안녕하세요! 20대 남성 직장인 분께서 요청하신 조건에 딱 맞는 신용카드를 꼼꼼히 분석해보았습니다.

---

## 1위 BEST 추천 카드: **LOCA 365 카드 (롯데카드)**  
🔗 [카드 상세정보 확인하기](https://www.card-gorilla.com/card/detail/2330)

### 이유  
- **관리비(아파트 관리비)** 자동납부 시 10% 할인 (월 최대 5,000원)  
- **가스비(도시가스비) 및 전기료** 자동납부 시 10% 할인 (월 최대 5,000원)  
- **대중교통(지하철, 시내버스)** 이용 시 10% 할인 (월 최대 5,000원)  
- **OTT 등 디지털 구독** (넷플릭스, 유튜브, 왓챠, 멜론, 지니뮤직, 디즈니 플러스) 월 1,500원 할인 제공  
- 각 할인은 기본적으로 전월 50만원 이상 이용 시 혜택 적용  
- 자동납부 및 공식 앱 결제 조건이 있으니 유의 필요  

### 혜택 세부내용  
- **아파트 관리비 자동납부 10% 할인**: 월 최대 5,000원, 2만원 이상 결제 건에 한함  
- **도시가스비, 전기료 자동납부 10% 할인**: 월 최대 5,000원, 2만원 이상 결제 건에 한함  
- **대중교통 10% 할인**: 지하철, 시내버스만 해당, 월 최대 5,000원, 2만원 이상 결제 건  
- **디지털 구독 서비스 1,500원 할인**: 넷플릭스, 유튜브 등 공식 홈페이지/앱에서 정기결제 시 월 1회 제공  
- 기타 배달앱(배달의민족, 요기요, 쿠팡이츠) 10% 할인 등도 추

In [ ]:
import time

# 1. 평가용 페르소나 5종 (혜택 차별화 버전)
evaluation_personas = [
    "20대 남성 직장인인데, 대중교통 이용이랑 OTT 구독, 그리고 관리비나 가스비 같은 공과금 할인이 다 되는 신용카드 좀 추천해줘.",
    "20대 여성 직장인인데, 퇴근 후 넷플릭스 보는 게 낙이라 OTT 할인이 꼭 필요해. 해외 직구를 많이 해서 결제 수수료도 아끼고 싶고, 온라인 간편결제가 되는 카드를 추천해줘",
    "10대 고등학생인데, 매일 버스랑 지하철로 등하교해서 대중교통 할인이 꼭 필요해요. OTT구독료도 아끼고 싶고, 온라인 간편결제 할인이 되는 체크카드 추천해주세요.",
    "30대 직장인인데, 지방 출장이 잦아서 전기차 충전 할인이 꼭 필요해. 고속도로를 탈 일이 많다 보니 통행료 할인도 있었으면 좋겠고, 이동 중에 편하게 들르는 드라이브스루 할인까지 지원되는 카드 추천해줘",
]

# 2. 평가 실행 및 저장 로직
filename = "model_evaluation_final.txt"

print("="*40)
print("📊 Human Based Evaluation 데이터 수집 시작")
print("조건: Temperature 0.8 / 페르소나별 10회 반복")
print("="*40)

with open(filename, "w", encoding="utf-8") as f:
    for p_idx, query in enumerate(evaluation_personas, 1):
        print(f"\n👤 페르소나 {p_idx} 진행 중...", end="")
        f.write(f"\n{'#'*30}\n")
        f.write(f"### [페르소나 {p_idx}] {query}\n")
        f.write(f"{'#'*30}\n\n")
        
        for run in range(1, 11):
            try:
                # RAG 실행 (gpt-5-nano, temp 0.8 적용된 ask 호출)
                response = concierge.ask(query)
                
                # 파일에 기록
                f.write(f"--- [RUN {run}/10] ---\n")
                f.write(response + "\n\n")
                
                print(".", end="", flush=True)
                time.sleep(1) # API 안정성을 위한 짧은 휴식
            except Exception as e:
                print(f"X (Error: {e})")
                f.write(f"--- [RUN {run}/10] 에러 발생: {e}\n\n")
        
        print(" 완료!")

print(f"\n\n✨ 모든 데이터 수집이 완료되었습니다!")
print(f"📄 생성된 파일: {filename}")
print("이제 이 파일을 열어 상/중/하 통계를 내시면 됩니다.")

📊 Human Based Evaluation 데이터 수집 시작
조건: Temperature 0.8 / 페르소나별 10회 반복

👤 페르소나 1 진행 중...추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
.추출된 키워드: ['아파트 관리비와 가스비 할인 혜택 포함 카드', '배달의민족 쿠팡이츠 배달 앱 할인 혜택 포함 카드']
. 완료!

👤 페르소나 2 진행 중...추출된 키워드: ['넷플릭스 유튜브 구독료 할인 혜택', '쿠팡 무신사 쇼핑 적립 혜택', '배달앱 주문 시 적립 혜택']
.추출된 키워드: ['넷플릭스 유튜브 구독료 할인 혜택', '쿠팡 무신사 쇼핑 적립 혜택', '배달앱 주문 시 적립 혜택']
.추출된 키워드: ['넷플릭스 유튜브 구독료 할인 혜택', '쿠팡 무신사 쇼핑 적립 혜택', 